## Download/Import Dependencies

In [ ]:
###### Install these when running for the first time  ######
# !pip install contractions
# !pip install mistletoe
# !pip install html2text
# !pip install -U -q PyDrive
# !pip install google-colab
# !pip install unrar
# !pip install --upgrade tensorflow-gpu
# !pip install tensorflow-hub
# !pip install sentence-transformers #install sentence transformer models


import re, string, unicodedata    
import contractions   
from bs4 import BeautifulSoup

from mistletoe import markdown
from html2text import HTML2Text

import numpy as np
import pandas as pd 

import nltk

# Uncomment and Download the below when running for the first time 
# nltk.download('stopwords')  
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('omw-1.4')


from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import matplotlib.pyplot as plt 
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

#to import files from google drive
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive
from google.colab import auth 
from oauth2client.client import GoogleCredentials
from google.colab import drive

import unrar

import tensorflow as tf
import tensorflow_hub as hub

from sentence_transformers import SentenceTransformer, util

from datetime import datetime as dt
import time

## Compress and upload the data to  Google Drive and Import the data from there

In [ ]:
# 1. Authorize Google Drive (required only for the first time, then comment it)
drive.mount('/content/gdrive', force_remount=True)

# 2. Compress the CSV files to .rar and upload the .rar file on your Google Drive (If compressing to .zip use zip commands instead)

# 3. Unrar the .rar file and put in the path after '-d' below
# !unrar x '/content/gdrive/My Drive/Colab Notebooks/ClassSpecsBLS.rar' -d '/content/gdrive/My Drive/Colab Notebooks/'

# 4. Read the CSV file from Google Drive saved in the above path
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ClassSpecsBLS.csv')
# xls = pd.ExcelFile('/content/gdrive/My Drive/Colab Notebooks/_Form.xlsx') 
# df = pd.read_csv('C:/Users/AdarshKuthuru/Downloads/UW_2022/NeoGov/ClassSpecsBLS.csv')
df.shape

Mounted at /content/gdrive


(800807, 14)

In [ ]:
df.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed
0,1,9,AGENCY JOB OPPORTUNITY,This class specification has been entered for ...,Varies by agency and posting.,Varies by agency and posting.,NaN,NaN,All employment offers are contingent upon succ...,"As an Approved Local Merit System, the County ...",Community and Social Service Occupations,"Counselors, Social Workers, and Other Communit...",Miscellaneous Community and Social Service Spe...,Probation Officers and Correctional Treatment ...
1,2,9,ELECTIONS COORDINATOR - SERVICES,"Under direction, to review, research, and anal...","• Reviews, researches, and analyzes elections ...","Knowledge of: Federal, State, and local laws, ...",NaN,Skill: Keyboard skill of 35 words per minute ...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Office and Administrative Support Occupations,Information and Record Clerks,Miscellaneous Information and Record Clerks,"Information and Record Clerks, All Other"
2,3,9,ELECTIONS COORDINATOR'S ASSISTANT,"Under general supervision, to assist an Electi...","• Researches, prepares, and recommends writte...","Knowledge of: Basic Federal, California State...",NaN,Skill: Keyboard skill to type 35 words per mi...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Office and Administrative Support Occupations,Information and Record Clerks,Miscellaneous Information and Record Clerks,"Information and Record Clerks, All Other"
3,4,9,ELECTIONS TECHNICIAN SUPERVISOR,"Under direction, to supervise a clerical funct...","• Supervises, assigns, instructs, and reviews ...",Knowledge of: Principles and techniques of sup...,NaN,Skill: Skill to type 35 words per minute may b...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Office and Administrative Support Occupations,Supervisors of Office and Administrative Suppo...,First-Line Supervisors of Office and Administr...,First-Line Supervisors of Office and Administr...
4,5,9,ELECTIONS TECHNICIAN III - SERVICES,To serve the voting and candidate public by pe...,"• Reviews reports, records, and other documen...",Knowledge of: Modern office practices and pro...,NaN,Skill: Sufficient keyboard or typing skill to...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Office and Administrative Support Occupations,Information and Record Clerks,Miscellaneous Information and Record Clerks,"Information and Record Clerks, All Other"


## Convert HTML to Text with formatting (Skip this)

In [ ]:
# sample = df[df['Class_Title'] == 'State\'s Attorney Investigator III']
# sample
# sample.to_csv("test1.csv")

In [ ]:
# sample.to_csv("Sample.csv")

# md = HTML2Text().handle(sample.iloc[0,3])
# html2 = markdown(md)
# text = BeautifulSoup(html2).getText()
# print(text)
# text

# test = BeautifulSoup(sample.iloc[0,3])
# test1 = test.get_text()
# print(test1)
# sample

## Subsetting the data for analysis: BLS_Detailed group = 'Nursing Assistants'

In [ ]:
df2 = df[df['BLS_Detailed']=='Nursing Assistants']
df2.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed
409,568,9,NURSING ASSISTANT,"<span style=""ignore-css: 11.0pt;""><span style=...","<br><span style=""ignore-css: 11.0pt;""><span st...","<span style=""ignore-css: 11.0pt;""><span style=...",NaN,"<span style=""ignore-css: 11.0pt;""><span style=...","<span style=""ignore-css: 10.0pt;""><span style=...","<span style=""ignore-css: 10.0pt;""><span style=...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants
410,569,9,ANESTHESIOLOGY TECHNICIAN,"Under general supervision, maintains an adequa...",• Maintain a complete stock of medications in ...,<strong><u>OPTION I<br></u></strong>Experience...,NaN,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants
412,571,9,NURSING ASSISTANT - PER DIEM,"Under supervision, assists professional nursin...",• Provide basic bedside nursing care to assure...,Education: Successful completion of the educat...,NaN,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants
702,966,9,PROFESSIONAL NURSING STUDENT INTERN,"Under supervision, to assist professional nurs...","<br>• Observes, collects and records assessmen...",Education: Current enrollment in a nursing pro...,NaN,NaN,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants
1718,2499,43,HOSPITAL TECHNICIAN TRAINEE,Under general supervision to perform a variety...,duties may include but are not limited to the ...,"Knowledge, Skills, and Abilities: Working abil...",Education/Experience: No experience or trainin...,NaN,NaN,NaN,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants


In [ ]:
# describing stats of variables
df2.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ClassSpecID,569.0,NaN,NaN,NaN,930419.481547,451984.98363,568.0,767622.0,1065409.0,1293422.0,1444832.0
EmployerID,569.0,NaN,NaN,NaN,3272.525483,2398.751569,9.0,1546.0,2213.0,5180.0,11487.0
Class_Title,569,475,Certified Nursing Assistant,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Class_Concept,533,495,<strong><u>Major Agencies:</u></strong><br>Men...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ExamplesOfDuties,517,478,"<ul style=""margin-bottom: 3pt;""><li>Provides p...",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MinimumQualifications,514,486,<ul>\t<li>Registered Professional Nurse with c...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SupplementalInfo,440,403,"<ul>\t<li>Ability to sit, stand, bend, and sto...",5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherRequirements,168,148,These physical requirements are not exhaustive...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherTitle1,125,108,All employment offers are contingent upon succ...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OtherTitle2,102,82,&nbsp;,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Count of unique values
# df2.Class_Title.nunique()
df2.ClassSpecID.nunique()

#check for nulls
df2.isnull().sum(axis=0)     

ClassSpecID                0
EmployerID                 0
Class_Title                0
Class_Concept             36
ExamplesOfDuties          52
MinimumQualifications     55
SupplementalInfo         129
OtherRequirements        401
OtherTitle1              444
OtherTitle2              467
BLS_Major                  0
BLS_Minor                  0
BLS_Broad                  0
BLS_Detailed               0
dtype: int64

## Data Pre-processing:

* Remove html tags and bullets.
* Replace contractions in string. (e.g. replace I'm --> I am) and so on.\
* Remove numbers.
* Tokenization
* To remove Stopwords.
* Lemmatized data
* We have used NLTK library to tokenize words , remove stopwords and lemmatize the remaining words.

In [ ]:
data = df2.copy()

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

data['Class_Concept'] = data['Class_Concept'].apply(lambda x: strip_html(str(x)))
data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: strip_html(str(x)))
data['MinimumQualifications'] = data['MinimumQualifications'].apply(lambda x: strip_html(str(x)))
data['SupplementalInfo'] = data['SupplementalInfo'].apply(lambda x: strip_html(str(x)))
data['OtherRequirements'] = data['OtherRequirements'].apply(lambda x: strip_html(str(x)))
data['OtherTitle1'] = data['OtherTitle1'].apply(lambda x: strip_html(str(x)))
data['OtherTitle2'] = data['OtherTitle2'].apply(lambda x: strip_html(str(x)))

#Remove all the bullets in the text
# data['Class_Concept'] = data['Class_Concept'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['MinimumQualifications'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['ExamplesOfDuties'] = data['ExamplesOfDuties'].apply(lambda x: str(x).replace('•', ''))
# data['OtherTitle1'] = data['OtherTitle1'].apply(lambda x: str(x).replace('•', ''))
# data['OtherTitle2'] = data['OtherTitle2'].apply(lambda x: str(x).replace('•', ''))

#Create Job Description column by merging Class_Concept, ExamplesOfDuties, MinimumQualifications, SupplementalInfo, OtherRequirements columns
data['Job_Desc'] = data[data.columns[3:9]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

data.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,Job_Desc
409,568,9,NURSING ASSISTANT,"Under supervision, assists professional nursin...",• Provide basic bedside nursing care to assure...,Education: Successful completion of the educat...,nan,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under supervision, assists professional nursin..."
410,569,9,ANESTHESIOLOGY TECHNICIAN,"Under general supervision, maintains an adequa...",• Maintain a complete stock of medications in ...,OPTION IExperience: One year in an acute care ...,nan,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under general supervision, maintains an adequa..."
412,571,9,NURSING ASSISTANT - PER DIEM,"Under supervision, assists professional nursin...",• Provide basic bedside nursing care to assure...,Education: Successful completion of the educat...,nan,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under supervision, assists professional nursin..."
702,966,9,PROFESSIONAL NURSING STUDENT INTERN,"Under supervision, to assist professional nurs...","• Observes, collects and records assessment da...",Education: Current enrollment in a nursing pro...,nan,nan,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under supervision, to assist professional nurs..."
1718,2499,43,HOSPITAL TECHNICIAN TRAINEE,Under general supervision to perform a variety...,duties may include but are not limited to the ...,"Knowledge, Skills, and Abilities: Working abil...",Education/Experience: No experience or trainin...,nan,nan,nan,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,Under general supervision to perform a variety...


In [ ]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

data['Job_Desc'] = data['Job_Desc'].apply(lambda x: replace_contractions(x))
data.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,Job_Desc
409,568,9,NURSING ASSISTANT,"Under supervision, assists professional nursin...",• Provide basic bedside nursing care to assure...,Education: Successful completion of the educat...,nan,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under supervision, assists professional nursin..."
410,569,9,ANESTHESIOLOGY TECHNICIAN,"Under general supervision, maintains an adequa...",• Maintain a complete stock of medications in ...,OPTION IExperience: One year in an acute care ...,nan,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under general supervision, maintains an adequa..."
412,571,9,NURSING ASSISTANT - PER DIEM,"Under supervision, assists professional nursin...",• Provide basic bedside nursing care to assure...,Education: Successful completion of the educat...,nan,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under supervision, assists professional nursin..."
702,966,9,PROFESSIONAL NURSING STUDENT INTERN,"Under supervision, to assist professional nurs...","• Observes, collects and records assessment da...",Education: Current enrollment in a nursing pro...,nan,nan,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under supervision, to assist professional nurs..."
1718,2499,43,HOSPITAL TECHNICIAN TRAINEE,Under general supervision to perform a variety...,duties may include but are not limited to the ...,"Knowledge, Skills, and Abilities: Working abil...",Education/Experience: No experience or trainin...,nan,nan,nan,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,Under general supervision to perform a variety...


# Create Pair-wise Master data

In [ ]:
# Subset the data
data2 = data[['ClassSpecID', 'EmployerID', 'Class_Title', 'Job_Desc']]
data2.fillna('', inplace=True)
data2.dropna(how='all', axis=1, inplace=True)
print(data2.shape)
data2.head()

(569, 4)


,ClassSpecID,EmployerID,Class_Title,Job_Desc
409,568,9,NURSING ASSISTANT,"Under supervision, assists professional nursin..."
410,569,9,ANESTHESIOLOGY TECHNICIAN,"Under general supervision, maintains an adequa..."
412,571,9,NURSING ASSISTANT - PER DIEM,"Under supervision, assists professional nursin..."
702,966,9,PROFESSIONAL NURSING STUDENT INTERN,"Under supervision, to assist professional nurs..."
1718,2499,43,HOSPITAL TECHNICIAN TRAINEE,Under general supervision to perform a variety...


In [ ]:
#performs a cross-join
pair_master2 = data2.merge(data2, how='cross') 

#rename columns
pair_master2 = pair_master2.rename(columns={'ClassSpecID_x':'ClassSpecID1','ClassSpecID_y':'ClassSpecID2','Class_Title_x':'Class_Title1','Class_Title_y':'Class_Title2','EmployerID_x':'EmployerID1','EmployerID_y':'EmployerID2',\
        'Job_Desc_x':'Job_Desc1','Job_Desc_y':'Job_Desc2'})

#rearrange columns and add pair_ID
pair_master2['Pair_ID'] = pair_master2['ClassSpecID1'].astype(str) + '-' + pair_master2['ClassSpecID2'].astype(str)
col_list = ['Pair_ID','ClassSpecID1','ClassSpecID2','Class_Title1','Class_Title2','EmployerID1','EmployerID2','Job_Desc1','Job_Desc2']
pair_master2 = pair_master2[col_list]
# pair_master2.to_csv("pair-master.csv")
print(pair_master2.shape)
pair_master2.head()

(323761, 9)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID1,EmployerID2,Job_Desc1,Job_Desc2
0,568-568,568,568,NURSING ASSISTANT,NURSING ASSISTANT,9,9,"Under supervision, assists professional nursin...","Under supervision, assists professional nursin..."
1,568-569,568,569,NURSING ASSISTANT,ANESTHESIOLOGY TECHNICIAN,9,9,"Under supervision, assists professional nursin...","Under general supervision, maintains an adequa..."
2,568-571,568,571,NURSING ASSISTANT,NURSING ASSISTANT - PER DIEM,9,9,"Under supervision, assists professional nursin...","Under supervision, assists professional nursin..."
3,568-966,568,966,NURSING ASSISTANT,PROFESSIONAL NURSING STUDENT INTERN,9,9,"Under supervision, assists professional nursin...","Under supervision, to assist professional nurs..."
4,568-2499,568,2499,NURSING ASSISTANT,HOSPITAL TECHNICIAN TRAINEE,9,43,"Under supervision, assists professional nursin...",Under general supervision to perform a variety...


## GT1 - About 5k Target Pairs

In [ ]:
sample_titles = ['Community Health Assistant','Medical Assistant II','Nursing Supervisor']
    
pair_master_5k = pair_master2[pair_master2['Class_Title1'].isin(sample_titles)]
print(pair_master_5k.shape)
pair_master_5k.head()

(5690, 9)


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID1,EmployerID2,Job_Desc1,Job_Desc2
113800,861670-568,861670,568,Medical Assistant II,NURSING ASSISTANT,2451,9,"DEFINITION Under general supervision, provide...","Under supervision, assists professional nursin..."
113801,861670-569,861670,569,Medical Assistant II,ANESTHESIOLOGY TECHNICIAN,2451,9,"DEFINITION Under general supervision, provide...","Under general supervision, maintains an adequa..."
113802,861670-571,861670,571,Medical Assistant II,NURSING ASSISTANT - PER DIEM,2451,9,"DEFINITION Under general supervision, provide...","Under supervision, assists professional nursin..."
113803,861670-966,861670,966,Medical Assistant II,PROFESSIONAL NURSING STUDENT INTERN,2451,9,"DEFINITION Under general supervision, provide...","Under supervision, to assist professional nurs..."
113804,861670-2499,861670,2499,Medical Assistant II,HOSPITAL TECHNICIAN TRAINEE,2451,43,"DEFINITION Under general supervision, provide...",Under general supervision to perform a variety...


# Estimate Similarity Scores

## 1. Universal Sentence Encoder (USE)

### 1.1 Load Universal Sentence Encoder (USE) model from TF-Hub

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


### 1.2. Compute the similarity between two sentences:

In [ ]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def cosine_similarity_USE(row):
    try:
        sentence1 = row.Job_Desc1
        sentence2 = row.Job_Desc2
        return round(float(cosine(model([sentence1])[0], model([sentence2])[0]) * 100), 2)
    except ValueError:
        return 0

In [ ]:
# row = pair_master_5k[pair_master_5k['Pair_ID'] == '861670-568']
# sentence1 = row.Job_Desc1
# sentence2 = row.Job_Desc2
# sentence1
# # cosine_similarity_USE(row)

In [ ]:
start_time = dt.now()
pair_master_5k["USE_sim"] = pair_master_5k.apply(cosine_similarity_USE, axis=1)
end_time = dt.now()
difference = end_time - start_time
print(f" Total time taken is {difference}")
pair_master_5k.head()

 Total time taken is 0:01:31.749294


,Pair_ID,ClassSpecID1,ClassSpecID2,Class_Title1,Class_Title2,EmployerID1,EmployerID2,Job_Desc1,Job_Desc2,USE_sim
113800,861670-568,861670,568,Medical Assistant II,NURSING ASSISTANT,2451,9,"DEFINITION Under general supervision, provide...","Under supervision, assists professional nursin...",78.33
113801,861670-569,861670,569,Medical Assistant II,ANESTHESIOLOGY TECHNICIAN,2451,9,"DEFINITION Under general supervision, provide...","Under general supervision, maintains an adequa...",73.62
113802,861670-571,861670,571,Medical Assistant II,NURSING ASSISTANT - PER DIEM,2451,9,"DEFINITION Under general supervision, provide...","Under supervision, assists professional nursin...",78.06
113803,861670-966,861670,966,Medical Assistant II,PROFESSIONAL NURSING STUDENT INTERN,2451,9,"DEFINITION Under general supervision, provide...","Under supervision, to assist professional nurs...",71.52
113804,861670-2499,861670,2499,Medical Assistant II,HOSPITAL TECHNICIAN TRAINEE,2451,43,"DEFINITION Under general supervision, provide...",Under general supervision to perform a variety...,70.68


In [ ]:
# import copy
# #creates a new dataframe
# new_df = pd.DataFrame() 

# for index, row in pair_master_5k.iterrows():
#     new_row = copy.deepcopy(row)
#     new_dict = new_row.to_dict()
    
#     new_dict['USE_sim'] = cosine(model([new_dict['Job_Desc1']])[0], model([new_dict['Job_Desc2']])[0])

#     new_df=new_df.append(pd.DataFrame(new_dict, index = [0])) #, ignore_index=True

# # new_df.to_csv('/content/gdrive/My Drive/Colab Notebooks/Data Output From Python.csv')
# new_df.head()
# type(list(pair_master_5k.iloc[0,7]))

## 2. SentenceTransformers (SBERT)
### 2.1 Load SentenceTransformers (SBERT) model 

In [ ]:
model = SentenceTransformer("nli-distilroberta-base-v2")

### 2.2. Compute the similarity between two sentences:

In [ ]:
def cosine_similarity_SBERT(row):
    sentence1 = row.Job_Desc1
    sentence2 = row.Job_Desc2
    embedding1 = model.encode([sentence1])
    embedding2 = model.encode([sentence2])

    #Compute cosine similarity between all pairs
    return round(float(util.cos_sim(embedding1, embedding2)[0][0] * 100), 2) 

In [ ]:
start_time = dt.now()
pair_master_5k["SBert_sim"] = pair_master_5k.apply(cosine_similarity_SBERT, axis=1)
end_time = dt.now()
difference = end_time - start_time
print(f" Total time taken is {difference}")

 Total time taken is 0:02:13.423262


## 3. Generate HTML Table

### 3.1. Clean missing values and create Job Description using raw data for BLS_detailed group - Nursing Assistants

In [ ]:
df2.fillna('', inplace=True)
df2['New_JD'] = df2['Class_Concept'] + "<br><br>" + df2['ExamplesOfDuties'] + "<br><br>" + df2['MinimumQualifications'] + "<br><br>" + df2['SupplementalInfo'] +\
 "<br><br>" + df2['OtherTitle1'] + "<br><br>" + df2['OtherTitle2']
df2.head()

,ClassSpecID,EmployerID,Class_Title,Class_Concept,ExamplesOfDuties,MinimumQualifications,SupplementalInfo,OtherRequirements,OtherTitle1,OtherTitle2,BLS_Major,BLS_Minor,BLS_Broad,BLS_Detailed,New_JD
409,568,9,NURSING ASSISTANT,"<span style=""ignore-css: 11.0pt;""><span style=...","<br><span style=""ignore-css: 11.0pt;""><span st...","<span style=""ignore-css: 11.0pt;""><span style=...",,"<span style=""ignore-css: 11.0pt;""><span style=...","<span style=""ignore-css: 10.0pt;""><span style=...","<span style=""ignore-css: 10.0pt;""><span style=...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"<span style=""ignore-css: 11.0pt;""><span style=..."
410,569,9,ANESTHESIOLOGY TECHNICIAN,"Under general supervision, maintains an adequa...",• Maintain a complete stock of medications in ...,<strong><u>OPTION I<br></u></strong>Experience...,,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under general supervision, maintains an adequa..."
412,571,9,NURSING ASSISTANT - PER DIEM,"Under supervision, assists professional nursin...",• Provide basic bedside nursing care to assure...,Education: Successful completion of the educat...,,License/Certificate: Possession of a valid Cal...,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under supervision, assists professional nursin..."
702,966,9,PROFESSIONAL NURSING STUDENT INTERN,"Under supervision, to assist professional nurs...","<br>• Observes, collects and records assessmen...",Education: Current enrollment in a nursing pro...,,,All employment offers are contingent upon succ...,"As an Approved Local Merit System, all County ...",Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,"Under supervision, to assist professional nurs..."
1718,2499,43,HOSPITAL TECHNICIAN TRAINEE,Under general supervision to perform a variety...,duties may include but are not limited to the ...,"Knowledge, Skills, and Abilities: Working abil...",Education/Experience: No experience or trainin...,,,,Healthcare Support Occupations,Home Health and Personal Care Aides; and Nursi...,"Nursing Assistants, Orderlies, and Psychiatric...",Nursing Assistants,Under general supervision to perform a variety...


### 3.2. Function to generate html file

In [ ]:
def pair_html(ClassSpecID1, ClassSpecID2):
  html_text = '<!doctype html> <html lang="en"> <head> <title>GT UI</title> <meta charset="utf-8"> <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">'+"<link href='https://fonts.googleapis.com/css?family=Roboto:400,100,300,700' rel='stylesheet' type='text/css'> " + '<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css"> <link rel="stylesheet" href="css\style.css" > </head> <style> table, th, td { border: 1px solid black; border-collapse: collapse; }, td:nth-child(even), th:nth-child(even) {background-color: #D6EEEE;} </style> <body> <section class="ftco-section"> <div class="container"> <div class="row justify-content-center"> <div class="col-md-6 text-center mb-5"> <h1 class="heading-section">GT UI</h1> </div> </div> <div class="row"> <div class="col-md-12"> <! <h4 class="text-center mb-4"> </h4> <div class="table-wrap"> <table class="table" style="height: 275px; width: 100%" border=1 frame=hsides rules=rows> <thead class="thead-primary"> <tr style="width: 100%; vertical-align:top"> <th>Pair ID</th> <th>Class Title-1</th> <th>Class Title-2</th> <th>Class Title Text Sim Score</th> <th>Job Desc-1</th> <th>Job Desc-2</th> <th>USE Sim Score</th> <th>SBERT Sim Score</th> </tr> </thead> <tbody> <tr style="vertical-align:top"> <td scope="row" class="scope" >' + str(str(ClassSpecID1)+"-"+ str(ClassSpecID2)) + '</td> <td>' + str(df[df["ClassSpecID"]==ClassSpecID1]["Class_Title"][0]) + '</td> <td>' + str(df[df["ClassSpecID"]==ClassSpecID2]["Class_Title"][1]) + '</td> <td>  </td> <td>'+ str(df[df["ClassSpecID"]==ClassSpecID1]["New_JD"][0]) + '</td> <td>' + str(df[df["ClassSpecID"]==ClassSpecID2]["New_JD"][1]) + '</td> <td>  </td> <td>  </td> </tr> </tbody> </table> </div> </div> </div> </div> </section> <script src="js/jquery.min.js"></script> <script src="js/popper.js"></script> <script src="js/bootstrap.min.js"></script> <script src="js/main.js"></script> </body> </html>'
  with open(str(ClassSpecID1)+"-"+ str(ClassSpecID2) + ".html", mode="w") as f:
    f.write(html_text)

In [ ]:
pair_html(568, 569)

KeyError: ignored